# 🌧️ Water Balance Analysis using Google Earth Engine + LSTM
This notebook demonstrates how to:
- Load Google Earth Engine data (rainfall, land cover, soil)
- Compute daily runoff using the SCS-CN method (already done in GEE)
- Export runoff time series for training an LSTM model in Python


In [ ]:
# Authenticate and initialize Earth Engine
import ee
ee.Authenticate()
ee.Initialize()

In [ ]:
# Define area of interest and time range
aoi = ee.Geometry.Rectangle([98.5, 18.0, 99.5, 19.5])
start = '2022-01-01'
end = '2022-12-31'

In [ ]:
# Load previously calculated runoff (e.g., from assets or computed)
runoff = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')\
  .filterDate(start, end)\
  .select('precipitation')\
  .map(lambda img: img.reduceRegion(
      reducer=ee.Reducer.mean(),
      geometry=aoi,
      scale=5500
  ).set({'date': img.date().format('YYYY-MM-dd')}))

In [ ]:
# Convert to Python list
runoff_list = runoff.aggregate_array('precipitation').getInfo()
dates = runoff.aggregate_array('date').getInfo()

## 📈 Prepare data for LSTM model
- Normalize
- Create time sequences
- Split to train/test

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# Convert and reshape
data = np.array(runoff_list).reshape(-1, 1)
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)